In [2]:
# from oficial python
import numpy as np
import matplotlib.pyplot as plt
from qutip import Bloch
from matplotlib import cm
import matplotlib as mpl
from qutip import basis, tensor
import time
from sklearn.model_selection import train_test_split

import pennylane as qml

# own modules:
from dll.spin_procesor_modificated import Quantum_Spin_Proces
from dll.main_fun import make_IRIS_v2
from dll. baseline_dru_v2 import Modelo_DRU
from dll.models import qcircuit_1_qubit_mixed
from dll.cost_functions import fidelity_cost
from dll.gate_procesor_modificated import CompositeGateProcessor

In [35]:
# Global
N_qubits = 1


# Base line parameters
N_layeres = 2
n_classes = 3
n_features = 3

# Spin processor parameters:



X, y =  make_IRIS_v2(n_features, n_classes)


model = Modelo_DRU(qcircuit_1_qubit_mixed, fidelity_cost,
                   epochs = 3, n_clases = n_classes, n_qubits = N_qubits,
                   features = n_features, alpha_noise = 0.0, save_process=False,
                   num_layers = N_layeres)

paramters_bef, bias_bef = model.params, model.bias

# splita data:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

# get parameters
paramters_1q, bias_1q = model.fit(X_train, y_train, X_test, y_test)

Epoch: 100%|██████████| 3/3 [00:02<00:00,  1.24it/s]


In [36]:
def calculate_fidelity(dm_bl, dm_spin):
    # Tratamiento de bl:
    dm_bl = np.array(dm_bl)
    nf, nc = dm_bl.shape
    if nf != nc:
        dm_bl = dm_bl @ dm_bl.T
    # tratamiento spin:
    nf, nc = dm_bl.shape
    if nf != nc:
        dm_bl = dm_bl @ dm_bl.T
    def my_fidelity(rho, sigma):
        sqrt_rho = np.sqrt(rho)
        fidelity_value = np.trace(np.sqrt(np.dot(np.dot(sqrt_rho, sigma), sqrt_rho))) ** 2
        return fidelity_value
    F = my_fidelity(dm_bl, dm_spin)
    return F

In [46]:
# Ideal base line
x_test_point = model.X_val[1]

states_base_line = qml.snapshots(qcircuit_1_qubit_mixed)(paramters_1q, x_test_point, bias=bias_1q)


# DRU in spin processor:
dru_circuit = CompositeGateProcessor(n_points_pulse_Ri=1000, n_points_pulse_Ri_spl = 100, version_qutip="5",
                                    N_qubits = N_qubits, tf_noise = False, tf_quantum_noise = False,
                                    ket_dru_0 = basis(2, 0), save_intra_states=True)         
                   
state_dru = dru_circuit.qcircuit_DRU_1_Qubit(paramters_1q, x_test_point, bias=bias_1q)

In [47]:
states_base_line

{'ket_1_qubits_0': tensor([[0.94075221+0.j        , 0.05715226-0.22906574j],
         [0.05715226+0.22906574j, 0.05924779+0.j        ]], requires_grad=True),
 'ket_1_qubits_1': tensor([[0.88515095-1.38777878e-17j, 0.21394177-2.36405727e-01j],
         [0.21394177+2.36405727e-01j, 0.11484905-1.38777878e-17j]], requires_grad=True),
 'execution_results': tensor([[0.88515095-1.38777878e-17j, 0.21394177-2.36405727e-01j],
         [0.21394177+2.36405727e-01j, 0.11484905-1.38777878e-17j]], requires_grad=True)}

In [48]:
dru_circuit.dict_states

{'ket_1_qubits_0': array([[0.75688905-0.60652386j],
        [0.19367244+0.14744072j]]),
 'ket_1_qubits_1': array([[0.5645366 -0.75262463j],
        [0.33746831-0.03114052j]])}

In [49]:
state_dru 

Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[0.5645366 -0.75262463j]
 [0.33746831-0.03114052j]]

In [50]:
dm_from_spins = (state_dru*state_dru.dag()).full()
dm_from_spins

array([[0.88514541+0.j      , 0.21395034-0.236407j],
       [0.21395034+0.236407j, 0.11485459+0.j      ]])

In [56]:
F  = qml.math.fidelity(dm_from_spins, states_base_line['execution_results'])
F

0.9999999998942763

In [51]:
def calculate_fidelity(dm_bl, dm_spin):
    # Tratamiento de bl:
    dm_bl = np.array(dm_bl)
    nf, nc = dm_bl.shape
    if nf != nc:
        dm_bl = dm_bl @ dm_bl.T
    # tratamiento spin:
    nf, nc = dm_spin.shape
    if nf != nc:
        dm_spin = dm_spin @ dm_spin.T
    return dm_bl, dm_spin 

In [52]:
a,b = calculate_fidelity(states_base_line['ket_1_qubits_0'], dru_circuit.dict_states['ket_1_qubits_0'])

In [53]:
def my_fidelity(rho, sigma):
        sqrt_rho = np.sqrt(rho)
        fidelity_value = np.trace(np.sqrt(np.dot(np.dot(sqrt_rho, sigma), sqrt_rho))) ** 2
        return fidelity_value

f = my_fidelity(a,b)
f

(0.8378311723324423-1.9253507586644905j)

In [54]:
qml.math.fidelity(a,b)

0.34170540608818806